In [2]:
['a', 'b', 'c'] + [1, 2, 3]

['a', 'b', 'c', 1, 2, 3]

In [2]:
type(lambda) 


SyntaxError: invalid syntax (<ipython-input-2-aad588dedfb0>, line 1)

In [7]:
[x**2 for x in range(10)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [4]:
import requests # to handle requests
import pandas as pd # for data analsysis
import numpy as np # to handle data in a vectorized manner

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
#tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

In [5]:
ClIENT_ID = '4FQQJE2LMGVLERRRE2BOFR1LQQR5ZEHFLTD1WRAFKOFT4TI0' # your Foursquare ID
ClIENT_SECRET = 'RCSNR5TH31NQRG2QYCHK101HRMTUZHBB2TSJ1CJU5UOGRXRQ' # your Foursquare Secret
VERSION = '20180604'
LIMIT =30
print('Your credentails:')
print('Foursquare_ID: ' + ClIENT_ID)
print('Foursquare_Secret:' + ClIENT_SECRET)

Your credentails:
Foursquare_ID: 4FQQJE2LMGVLERRRE2BOFR1LQQR5ZEHFLTD1WRAFKOFT4TI0
Foursquare_Secret:RCSNR5TH31NQRG2QYCHK101HRMTUZHBB2TSJ1CJU5UOGRXRQ


In [25]:
# define the city and get its latitude & longitude 
city = 'Banglore'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

18.0056901 79.5715088


In [36]:

# search for hotels
search_query = 'Hotel'
radius = 500

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=4FQQJE2LMGVLERRRE2BOFR1LQQR5ZEHFLTD1WRAFKOFT4TI0&client_secret=RCSNR5TH31NQRG2QYCHK101HRMTUZHBB2TSJ1CJU5UOGRXRQ&ll=18.0056901,79.5715088&v=20180604&query=Hotel&radius=500&limit=30'

In [43]:
# Send the GET Request and examine the results
results = requests.get(url).json()
#print(results)

In [41]:

# assign relevant part of JSON to venues
venues = results['response']['venues']
#print(venues)

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

<ipython-input-41-d61003b2e992>:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.formattedAddress,location.address,location.city,location.state,location.crossStreet,location.postalCode
0,550ee16d498e2a7e905403da,Hotel 🏩 Geetha Bhavan Veg,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",v-1599286295,False,18.005682,79.570202,"[{'label': 'display', 'lat': 18.00568151631746...",138,IN,India,[India],NaN,NaN,NaN,NaN,NaN
1,5294367011d2f190312cdf2e,Hotel Amaravathy,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1599286295,False,18.005700,79.571600,"[{'label': 'display', 'lat': 18.0057, 'lng': 7...",9,IN,India,"[Hanmakonda Chow rastha, Warangal, Telangana, ...",Hanmakonda Chow rastha,Warangal,Telangana,NaN,NaN
2,54c229c2498e205c485f6044,Supreme Hotel And Loadges,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1599286295,False,18.005431,79.571069,"[{'label': 'display', 'lat': 18.00543120534612...",54,IN,India,"[Near Bus Stko (Hanmakonda), Warangal, Telanga...",Near Bus Stko,Warangal,Telangana,Hanmakonda,NaN
3,51d95d19498ebd04ea216cc6,Hotel Ashoka,[],v-1599286295,False,18.006702,79.569396,"[{'label': 'display', 'lat': 18.00670236984074...",250,IN,India,"[Hanamkonda, Andhrapradesh, India]",NaN,Hanamkonda,Andhrapradesh,NaN,NaN
4,5d8c25fcfdbdcb00078a7b60,Hotel Radhika Oyo,"[{'id': '4bf58dd8d48988d1fb931735', 'name': 'M...",v-1599286295,False,18.003707,79.573492,"[{'label': 'display', 'lat': 18.003707, 'lng':...",304,IN,India,"[Hanamkonda 506011, TG, India]",NaN,Hanamkonda,TG,NaN,506011


In [39]:
# keep only columns that include venue name, and anything that is associated with location
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe.head()

,name,categories,lat,lng,labeledLatLngs,distance,cc,country,formattedAddress,address,city,state,crossStreet,postalCode,id
0,Hotel 🏩 Geetha Bhavan Veg,Asian Restaurant,18.005682,79.570202,"[{'label': 'display', 'lat': 18.00568151631746...",138,IN,India,[India],NaN,NaN,NaN,NaN,NaN,550ee16d498e2a7e905403da
1,Hotel Amaravathy,Indian Restaurant,18.005700,79.571600,"[{'label': 'display', 'lat': 18.0057, 'lng': 7...",9,IN,India,"[Hanmakonda Chow rastha, Warangal, Telangana, ...",Hanmakonda Chow rastha,Warangal,Telangana,NaN,NaN,5294367011d2f190312cdf2e
2,Supreme Hotel And Loadges,Hotel,18.005431,79.571069,"[{'label': 'display', 'lat': 18.00543120534612...",54,IN,India,"[Near Bus Stko (Hanmakonda), Warangal, Telanga...",Near Bus Stko,Warangal,Telangana,Hanmakonda,NaN,54c229c2498e205c485f6044
3,Hotel Ashoka,None,18.006702,79.569396,"[{'label': 'display', 'lat': 18.00670236984074...",250,IN,India,"[Hanamkonda, Andhrapradesh, India]",NaN,Hanamkonda,Andhrapradesh,NaN,NaN,51d95d19498ebd04ea216cc6
4,Hotel Radhika Oyo,Motel,18.003707,79.573492,"[{'label': 'display', 'lat': 18.003707, 'lng':...",304,IN,India,"[Hanamkonda 506011, TG, India]",NaN,Hanamkonda,TG,NaN,506011,5d8c25fcfdbdcb00078a7b60


In [45]:

# import Libraries that we need
import requests
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(page.content, 'html.parser')

In [47]:
print(page)

<Response [200]>


In [48]:
print(soup)

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"dcacf011-3e89-4a5d-819c-21afa04af03e","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":975466835,"wgRevisionId":975466835,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Communications in Ontario","

In [49]:
table = soup.find('tbody')
rows = table.select('tr')
row = [r.get_text() for r in rows]

In [50]:
print(table)b

<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td>North York
</td>
<td>Don Mills
</t

In [52]:
print(row)

['\nPostal Code\n\nBorough\n\nNeighbourhood\n', '\nM1A\n\nNot assigned\n\nNot assigned\n', '\nM2A\n\nNot assigned\n\nNot assigned\n', '\nM3A\n\nNorth York\n\nParkwoods\n', '\nM4A\n\nNorth York\n\nVictoria Village\n', '\nM5A\n\nDowntown Toronto\n\nRegent Park, Harbourfront\n', '\nM6A\n\nNorth York\n\nLawrence Manor, Lawrence Heights\n', "\nM7A\n\nDowntown Toronto\n\nQueen's Park, Ontario Provincial Government\n", '\nM8A\n\nNot assigned\n\nNot assigned\n', '\nM9A\n\nEtobicoke\n\nIslington Avenue, Humber Valley Village\n', '\nM1B\n\nScarborough\n\nMalvern, Rouge\n', '\nM2B\n\nNot assigned\n\nNot assigned\n', '\nM3B\n\nNorth York\n\nDon Mills\n', '\nM4B\n\nEast York\n\nParkview Hill, Woodbine Gardens\n', '\nM5B\n\nDowntown Toronto\n\nGarden District, Ryerson\n', '\nM6B\n\nNorth York\n\nGlencairn\n', '\nM7B\n\nNot assigned\n\nNot assigned\n', '\nM8B\n\nNot assigned\n\nNot assigned\n', '\nM9B\n\nEtobicoke\n\nWest Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale\n', '\nM1C\n\

In [55]:
df=pd.DataFrame(row)
df.head()

,0
0,\nPostal Code\n\nBorough\n\nNeighbourhood\n
1,\nM1A\n\nNot assigned\n\nNot assigned\n
2,\nM2A\n\nNot assigned\n\nNot assigned\n
3,\nM3A\n\nNorth York\n\nParkwoods\n
4,\nM4A\n\nNorth York\n\nVictoria Village\n


In [68]:
df = pd.DataFrame(row)
df1 = df[0].str.split('\n', expand=True)
df2 = df1.rename(columns=df1.iloc[0])
df3 = df2.drop(df2.index[0])
df4 = df3[df3.Borough != 'Not assigned']
df5 = df4.groupby(['Postal Code', 'Borough'], sort = False).agg(','.join)
df5.reset_index(inplace = True)
df6 = df5.replace("Not assigned", "Queen's Park")
df6

,Postal Code,Borough,,,,Neighbourhood,
0,M3A,North York,,,,Parkwoods,
1,M4A,North York,,,,Victoria Village,
2,M5A,Downtown Toronto,,,,"Regent Park, Harbourfront",
3,M6A,North York,,,,"Lawrence Manor, Lawrence Heights",
4,M7A,Downtown Toronto,,,,"Queen's Park, Ontario Provincial Government",
5,M9A,Etobicoke,,,,"Islington Avenue, Humber Valley Village",
6,M1B,Scarborough,,,,"Malvern, Rouge",
7,M3B,North York,,,,Don Mills,
8,M4B,East York,,,,"Parkview Hill, Woodbine Gardens",
9,M5B,Downtown Toronto,,,,"Garden District, Ryerson",
